In [3]:
pip install pyppeteer pyppeteer_stealth pandas tqdm nest_asyncio

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import uuid
import time
from datetime import datetime
import asyncio
import nest_asyncio
from pyppeteer import launch
from pyppeteer_stealth import stealth
import re
import os
from tqdm import tqdm

nest_asyncio.apply()

CITIES_AVAILABLE = ['bangalore', 'mumbai', 'hyderabad', 'ncr', 'pune', 'goa']

async def auto_scroll(page):
    await page.evaluate("""
        async function () {
            await new Promise((resolve) => {
                let totalHeight = 0;
                const distance = 500;
                const timer = setInterval(() => {
                    const scrollHeight = document.body.scrollHeight;
                    window.scrollBy(0, distance);
                    totalHeight += distance;

                    if (totalHeight >= scrollHeight){
                        clearInterval(timer);
                        resolve();
                    }
                }, 300);
            });
        }
    """)

async def safe_goto(page, url, retries=3):
    for attempt in range(retries):
        try:
            await page.goto(url, {'waitUntil': 'networkidle2', 'timeout': 60000})
            return True
        except Exception as e:
            print(f"⚠️ Retry {attempt+1}/{retries} for {url} failed: {e}")
            await asyncio.sleep(2 ** attempt)
    print(f"❌ Failed to load {url}")
    return False

async def extract_event_details(card):
    try:
        href = await card.querySelectorEval('a', 'el => el.href')
        name = await card.querySelectorEval('.slider-title', 'el => el.innerText')
        image_url = await card.querySelectorEval('img', 'el => el.src')
        date = await card.querySelectorEval('.formatted-date-span', 'el => el.innerText')
        time_ = await card.querySelectorEval('.formatted-day-span', 'el => el.innerText')
        location = await card.querySelectorEval('.slider-location', 'el => el.innerText')
        price = await card.querySelectorEval('.money-detail-section span', 'el => el.innerText')

        try:
            interested = await card.querySelectorEval('.icon-interested', 'el => el.innerText')
        except:
            interested = ''

        categories = await card.querySelectorAllEval('.cats span', 'nodes => nodes.map(n => n.innerText)')

        return {
            'event_name': name.strip(),
            'event_link': href.strip(),
            'image_url': image_url.strip(),
            'date': date.strip(),
            'time': time_.strip(),
            'location': location.strip(),
            'categories': ', '.join([c.strip() for c in categories]),
            'price_range': price.strip(),
            'interested_count': interested.strip()
        }

    except Exception as e:
        print(f"❌ Error extracting event card: {e}")
        return None

async def get_event_details_from_city(browser, city):
    city_url = f"https://highape.com/{city}/all-events"
    page = await browser.newPage()

    await page.setUserAgent('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
    await stealth(page)  # 🕵️ Apply stealth to bypass bot checks

    loaded = await safe_goto(page, city_url)
    if not loaded:
        await page.close()
        return []

    await asyncio.sleep(6)
    await auto_scroll(page)
    await asyncio.sleep(6)

    print(f"🔍 Extracting events for {city.title()}...")
    event_cards = await page.querySelectorAll('div.fonts.cards_div')
    print(f"📦 Found {len(event_cards)} event cards in {city.title()}")

    city_events = []
    for card in tqdm(event_cards, desc=f"{city.title()} Events", leave=False):
        event = await extract_event_details(card)
        if event:
            event['city'] = city.title()
            city_events.append(event)

    await page.close()
    return city_events

async def main():
    print("📍 Available cities:", ", ".join(CITIES_AVAILABLE))
    selected = input("👉 Enter city name(s) (comma-separated) or type 'all': ").strip().lower()

    if selected == 'all':
        cities = CITIES_AVAILABLE
    else:
        cities = [c.strip() for c in selected.split(',') if c.strip() in CITIES_AVAILABLE]

    if not cities:
        print("❌ No valid cities selected.")
        return

    browser = await launch(headless=True, args=['--no-sandbox'])
    all_events = []

    for city in tqdm(cities, desc="🌆 Scraping Cities"):
        city_events = await get_event_details_from_city(browser, city)
        all_events.extend(city_events)

    await browser.close()

    if not all_events:
        print("⚠️ No events scraped. Exiting.")
        return

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    cities_str = "_".join(cities)
    filename = f"highape_all_events_{cities_str}_{timestamp}.csv"
    pd.DataFrame(all_events).to_csv(filename, index=False)

    print(f"\n✅ Scraped {len(all_events)} events from {len(cities)} city(ies).")
    print(f"📁 Saved to: {filename}")

if __name__ == '__main__':
    asyncio.get_event_loop().run_until_complete(main())


📍 Available cities: bangalore, mumbai, hyderabad, ncr, pune, goa


👉 Enter city name(s) (comma-separated) or type 'all':  all


🌆 Scraping Cities:   0%|                                 | 0/6 [00:00<?, ?it/s]

⚠️ Retry 1/3 for https://highape.com/bangalore/all-events failed: Navigation Timeout Exceeded: 60000 ms exceeded.
⚠️ Retry 2/3 for https://highape.com/bangalore/all-events failed: Protocol Error (Page.navigate): Session closed. Most likely the page has been closed.
⚠️ Retry 3/3 for https://highape.com/bangalore/all-events failed: Protocol Error (Page.navigate): Session closed. Most likely the page has been closed.


🌆 Scraping Cities:   0%|                                 | 0/6 [01:07<?, ?it/s]

❌ Failed to load https://highape.com/bangalore/all-events


PageError: Protocol Error: Connection Closed. Most likely the page has been closed.

In [1]:
import pandas as pd
import uuid
import time
from datetime import datetime
import asyncio
import nest_asyncio
from pyppeteer import launch
from pyppeteer_stealth import stealth
import re
import os
from tqdm import tqdm

nest_asyncio.apply()

CITIES_AVAILABLE = ['bangalore', 'mumbai', 'hyderabad', 'ncr', 'pune', 'goa']
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'

async def auto_scroll(page):
    await page.evaluate("""
        async function () {
            await new Promise((resolve) => {
                let totalHeight = 0;
                const distance = 500;
                const timer = setInterval(() => {
                    const scrollHeight = document.body.scrollHeight;
                    window.scrollBy(0, distance);
                    totalHeight += distance;

                    if (totalHeight >= scrollHeight){
                        clearInterval(timer);
                        resolve();
                    }
                }, 300);
            });
        }
    """)

async def safe_goto(page, url, retries=3):
    for attempt in range(retries):
        try:
            await page.goto(url, {'waitUntil': 'networkidle2', 'timeout': 90000})
            return True
        except Exception as e:
            print(f"⚠️ Retry {attempt+1}/{retries} for {url} failed: {e}")
            await asyncio.sleep(2 ** attempt)
    print(f"❌ Failed to load {url}")
    return False

async def extract_event_details(card):
    try:
        href = await card.querySelectorEval('a', 'el => el.href')
        name = await card.querySelectorEval('.slider-title', 'el => el.innerText')
        image_url = await card.querySelectorEval('img', 'el => el.src')
        date = await card.querySelectorEval('.formatted-date-span', 'el => el.innerText')
        time_ = await card.querySelectorEval('.formatted-day-span', 'el => el.innerText')
        location = await card.querySelectorEval('.slider-location', 'el => el.innerText')
        price = await card.querySelectorEval('.money-detail-section span', 'el => el.innerText')
        try:
            interested = await card.querySelectorEval('.icon-interested', 'el => el.innerText')
        except:
            interested = ''
        categories = await card.querySelectorAllEval('.cats span', 'nodes => nodes.map(n => n.innerText)')
        return {
            'event_name': name.strip(),
            'event_link': href.strip(),
            'image_url': image_url.strip(),
            'date': date.strip(),
            'time': time_.strip(),
            'location': location.strip(),
            'categories': ', '.join([c.strip() for c in categories]),
            'price_range': price.strip(),
            'interested_count': interested.strip()
        }
    except Exception as e:
        print(f"❌ Error extracting event card: {e}")
        return None

async def get_event_details_from_city(browser, city):
    city_url = f"https://highape.com/{city}/all-events"
    page = None
    try:
        page = await browser.newPage()
        await page.setUserAgent(USER_AGENT)
        await stealth(page)

        loaded = await safe_goto(page, city_url)
        if not loaded:
            # await page.screenshot({'path': f"{city}_failed_load.png", 'fullPage': True})  # Optional visual debug
            return []

        await asyncio.sleep(6)
        await auto_scroll(page)
        await asyncio.sleep(6)

        print(f"🔍 Extracting events for {city.title()}...")
        event_cards = await page.querySelectorAll('div.fonts.cards_div')
        print(f"📦 Found {len(event_cards)} event cards in {city.title()}")

        city_events = []
        for card in tqdm(event_cards, desc=f"{city.title()} Events", leave=False):
            event = await extract_event_details(card)
            if event:
                event['city'] = city.title()
                city_events.append(event)

        return city_events

    except Exception as e:
        print(f"❌ Unexpected error in city {city}: {e}")
        return []

    finally:
        if page:
            try:
                await page.close()
            except Exception as e:
                print(f"⚠️ Could not close page for {city}: {e}")

async def main():
    print("📍 Available cities:", ", ".join(CITIES_AVAILABLE))
    selected = input("👉 Enter city name(s) (comma-separated) or type 'all': ").strip().lower()

    if selected == 'all':
        cities = CITIES_AVAILABLE
    else:
        cities = [c.strip() for c in selected.split(',') if c.strip() in CITIES_AVAILABLE]

    if not cities:
        print("❌ No valid cities selected.")
        return

    browser = await launch(
        headless=True,
        args=[
            '--no-sandbox',
            '--disable-setuid-sandbox',
            '--disable-gpu',
            '--disable-dev-shm-usage',
            '--disable-extensions',
            '--start-maximized',
            '--single-process',
            '--no-zygote'
        ]
    )

    all_events = []
    for city in tqdm(cities, desc="🌆 Scraping Cities"):
        city_events = await get_event_details_from_city(browser, city)
        all_events.extend(city_events)

    await browser.close()

    if not all_events:
        print("⚠️ No events scraped. Exiting.")
        return

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    cities_str = "_".join(cities)
    filename = f"highape_all_events_{cities_str}_{timestamp}.csv"
    pd.DataFrame(all_events).to_csv(filename, index=False)

    print(f"\n✅ Scraped {len(all_events)} events from {len(cities)} city(ies).")
    print(f"📁 Saved to: {filename}")

if __name__ == '__main__':
    asyncio.get_event_loop().run_until_complete(main())


📍 Available cities: bangalore, mumbai, hyderabad, ncr, pune, goa


👉 Enter city name(s) (comma-separated) or type 'all':  bangalore


🌆 Scraping Cities:   0%|                                 | 0/1 [00:00<?, ?it/s]

⚠️ Retry 1/3 for https://highape.com/bangalore/all-events failed: Navigation Timeout Exceeded: 90000 ms exceeded.
⚠️ Retry 2/3 for https://highape.com/bangalore/all-events failed: Protocol Error (Page.navigate): Session closed. Most likely the page has been closed.
⚠️ Retry 3/3 for https://highape.com/bangalore/all-events failed: Protocol Error (Page.navigate): Session closed. Most likely the page has been closed.


🌆 Scraping Cities: 100%|█████████████████████████| 1/1 [01:37<00:00, 97.03s/it]

❌ Failed to load https://highape.com/bangalore/all-events
⚠️ Could not close page for bangalore: Protocol Error: Connection Closed. Most likely the page has been closed.
⚠️ No events scraped. Exiting.


In [15]:
import pandas as pd
from datetime import datetime
import asyncio
import nest_asyncio
from pyppeteer import launch
from pyppeteer_stealth import stealth
from tqdm import tqdm

nest_asyncio.apply()

CITIES_AVAILABLE = ['bangalore', 'mumbai', 'hyderabad', 'ncr', 'pune', 'goa']
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'

async def auto_scroll(page):
    await page.evaluate("""
        async function () {
            await new Promise((resolve) => {
                let totalHeight = 0;
                const distance = 500;
                const timer = setInterval(() => {
                    const scrollHeight = document.body.scrollHeight;
                    window.scrollBy(0, distance);
                    totalHeight += distance;

                    if (totalHeight >= scrollHeight){
                        clearInterval(timer);
                        resolve();
                    }
                }, 300);
            });
        }
    """)

async def safe_goto(page, url, retries=3):
    for attempt in range(retries):
        try:
            await page.goto(url, {'waitUntil': 'networkidle2', 'timeout': 90000})
            return True
        except Exception as e:
            print(f"⚠️ Retry {attempt+1}/{retries} for {url} failed: {e}")
            await asyncio.sleep(2 ** attempt)
    print(f"❌ Failed to load {url}")
    return False

async def extract_event_details(card):
    try:
        href = await card.querySelectorEval('a', 'el => el.href')
        name = await card.querySelectorEval('.slider-title', 'el => el.innerText')
        image_url = await card.querySelectorEval('img', 'el => el.src')
        date = await card.querySelectorEval('.formatted-date-span', 'el => el.innerText')
        time_ = await card.querySelectorEval('.formatted-day-span', 'el => el.innerText')
        location = await card.querySelectorEval('.slider-location', 'el => el.innerText')
        price = await card.querySelectorEval('.money-detail-section span', 'el => el.innerText')
        try:
            interested = await card.querySelectorEval('.icon-interested', 'el => el.innerText')
        except:
            interested = ''
        categories = await card.querySelectorAllEval('.cats span', 'nodes => nodes.map(n => n.innerText)')
        return {
            'event_name': name.strip(),
            'event_link': href.strip(),
            'image_url': image_url.strip(),
            'date': date.strip(),
            'time': time_.strip(),
            'location': location.strip(),
            'categories': ', '.join([c.strip() for c in categories]),
            'price_range': price.strip(),
            'interested_count': interested.strip()
        }
    except Exception as e:
        print(f"❌ Error extracting event card: {e}")
        return None

async def get_event_details_from_city(browser, city):
    city_url = f"https://highape.com/{city}/all-events"
    page = None
    try:
        page = await browser.newPage()
        await page.setUserAgent(USER_AGENT)
        await stealth(page)

        loaded = await safe_goto(page, city_url)
        if not loaded:
            return []

        await asyncio.sleep(6)
        await auto_scroll(page)
        await asyncio.sleep(6)

        print(f"🔍 Extracting events for {city.title()}...")
        event_cards = await page.querySelectorAll('div.fonts.cards_div')
        print(f"📦 Found {len(event_cards)} event cards in {city.title()}")

        city_events = []
        for card in tqdm(event_cards, desc=f"{city.title()} Events", leave=False):
            event = await extract_event_details(card)
            if event:
                event['city'] = city.title()
                city_events.append(event)

        return city_events

    except Exception as e:
        print(f"❌ Unexpected error in city {city}: {e}")
        return []

    finally:
        if page:
            try:
                await page.close()
            except Exception as e:
                print(f"⚠️ Could not close page for {city}: {e}")

async def main():
    print("📍 Available cities:", ", ".join(CITIES_AVAILABLE))
    selected = input("👉 Enter city name(s) (comma-separated) or type 'all': ").strip().lower()

    if selected == 'all':
        cities = CITIES_AVAILABLE
    else:
        cities = [c.strip() for c in selected.split(',') if c.strip() in CITIES_AVAILABLE]

    if not cities:
        print("❌ No valid cities selected.")
        return

    browser = await launch(
        headless=False,  # 👈 Running with browser window open
        args=['--no-sandbox', '--disable-setuid-sandbox']
    )

    all_events = []
    for city in tqdm(cities, desc="🌆 Scraping Cities"):
        city_events = await get_event_details_from_city(browser, city)
        all_events.extend(city_events)

    await browser.close()

    if not all_events:
        print("⚠️ No events scraped. Exiting.")
        return

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    cities_str = "_".join(cities)
    filename = f"highape_all_events_{cities_str}_{timestamp}.csv"
    pd.DataFrame(all_events).to_csv(filename, index=False)

    print(f"\n✅ Scraped {len(all_events)} events from {len(cities)} city(ies).")
    print(f"📁 Saved to: {filename}")

if __name__ == '__main__':
    asyncio.get_event_loop().run_until_complete(main())


📍 Available cities: bangalore, mumbai, hyderabad, ncr, pune, goa


👉 Enter city name(s) (comma-separated) or type 'all':  all


🌆 Scraping Cities:   0%|                                 | 0/6 [00:00<?, ?it/s]

🔍 Extracting events for Bangalore...
📦 Found 296 event cards in Bangalore



🌆 Scraping Cities:  17%|████▏                    | 1/6 [00:39<03:15, 39.08s/it]

...

🔍 Extracting events for Pune...
📦 Found 1 event cards in Pune



🌆 Scraping Cities:  83%|████████████████████▊    | 5/6 [01:46<00:18, 18.83s/it]

🔍 Extracting events for Goa...
📦 Found 29 event cards in Goa



🌆 Scraping Cities: 100%|█████████████████████████| 6/6 [02:05<00:00, 20.89s/it]



✅ Scraped 356 events from 6 city(ies).
📁 Saved to: highape_all_events_bangalore_mumbai_hyderabad_ncr_pune_goa_20250607_143542.csv


In [ ]:
import pandas as pd
import uuid
import time
from datetime import datetime
import json
import re
import asyncio
import nest_asyncio
from pyppeteer import launch
import sys

nest_asyncio.apply()

async def auto_scroll(page):
    await page.evaluate("""
        async function () {
            await new Promise((resolve) => {
                let totalHeight = 0;
                const distance = 500;
                const timer = setInterval(() => {
                    const scrollHeight = document.body.scrollHeight;
                    window.scrollBy(0, distance);
                    totalHeight += distance;

                    if (totalHeight >= scrollHeight){
                        clearInterval(timer);
                        resolve();
                    }
                }, 300);
            });
        }
    """)

async def extract_event_details(card):
    try:
        href = await card.querySelectorEval('a', 'el => el.href')
        name = await card.querySelectorEval('.slider-title', 'el => el.innerText')
        image_url = await card.querySelectorEval('img', 'el => el.src')
        date = await card.querySelectorEval('.formatted-date-span', 'el => el.innerText')
        time = await card.querySelectorEval('.formatted-day-span', 'el => el.innerText')
        location = await card.querySelectorEval('.slider-location', 'el => el.innerText')
        price = await card.querySelectorEval('.money-detail-section span', 'el => el.innerText')
        try:
            interested = await card.querySelectorEval('.icon-interested', 'el => el.innerText')
        except:
            interested = ''
        categories = await card.querySelectorAllEval('.cats span', 'nodes => nodes.map(n => n.innerText)')
        return {
            'event_name': name,
            'event_link': href,
            'image_url': image_url,
            'date': date,
            'time': time,
            'location': location,
            'categories': ', '.join(categories),
            'price_range': price,
            'interested_count': interested
        }
    except Exception as e:
        print(f"❌ Error extracting details: {e}")
        return None

async def get_event_details_from_city(browser, city):
    city_url = f"https://highape.com/{city}/all-events"
    page = await browser.newPage()
    await page.setUserAgent('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
    await page.goto(city_url, {'waitUntil': 'networkidle2', 'timeout': 60000})
    await asyncio.sleep(10)
    await auto_scroll(page)
    await asyncio.sleep(10)

    print(f"🔍 Extracting events for {city.title()}...")
    event_cards = await page.querySelectorAll('div.fonts.cards_div')
    print(f"📦 Found {len(event_cards)} event cards in {city.title()}")

    city_events = []
    for card in event_cards:
        event = await extract_event_details(card)
        if event:
            event['city'] = city.title()
            city_events.append(event)

    await page.close()
    return city_events

async def main():
    cities = ['bangalore', 'mumbai', 'hyderabad', 'ncr', 'pune', 'goa']
    browser = await launch(headless=False, args=['--no-sandbox'])
    all_events = []

    for city in cities:
        city_events = await get_event_details_from_city(browser, city)
        all_events.extend(city_events)

    await browser.close()

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"highape_all_events_{timestamp}.csv"
    df = pd.DataFrame(all_events)
    df.to_csv(filename, index=False)

    print(f"✅ All events saved to {filename}")

if __name__ == '__main__':
    asyncio.get_event_loop().run_until_complete(main())

In [ ]:
import pandas as pd
import uuid
import time
from datetime import datetime
import json
import re
import asyncio
import nest_asyncio
from pyppeteer import launch
import sys

nest_asyncio.apply()

def create_empty_event():
    return {
        'externalEventLink': '',
        'eventId': str(uuid.uuid4()),
        'plurEventLink': '',
        'eventName': '',
        'eventDescription': '',
        'eventDateAndTime': '',
        'eventDuration': '',
        'venueName': '',
        'locality': '',
        'address': '',
        'zipcode': '',
        'lat': '',
        'lon': '',
        'ticketAmount': '',
        'guestlistInfo': '',
        'ticketLink': '',
        'supportedLanguages': '',
        'category': '',
        'recurrence': '',
        'attendeesCount': '',
        'joinChatDetails': '',
        'policyAndConditions': '',
        'frequentlyAskedQuestions': '',
        'city': '',
        'state': '',
        'geolocation': '',
        'isRecurring': 'No',
        'pattern': '',
        'eventType': '',
        'subCategory': '',
        'venueId': '',
        'foodAvailable': '',
        'smokingAllowed': '',
        'wheelchairAccess': '',
        'parkingAvailable': '',
        'supportAvailable': '',
        'layout': '',
        'petFriendly': False,
        'alcoholServed': '',
        'minimumAge': '',
        'ticketsAtVenue': '',
        'prohibitedItemList': '',
        'washroomAvailable': '',
        'onlineMenu': '',
        'happyHours': '',
        'danceFloorAvailable': '',
        'poolAvailable': '',
        'isActivityAvailable': '',
        'artists': '',
        'highlightImages': '',
        'highlightVideos': '',
        'galleryImages': '',
        'galleryVideos': '',
        'kidFriendly': False,
        'seatingArrangement': '',
        'entryAllowedFor': '',
        'tags': ''
    }

async def scrape_highape_event(page, url):
    event = create_empty_event()
    event['externalEventLink'] = url

    try:
        await page.setUserAgent('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
        await page.setViewport({'width': 1200, 'height': 800})
        await page.goto(url, {'waitUntil': 'networkidle2', 'timeout': 60000})
        await asyncio.sleep(5)

        try:
            event['eventName'] = await page.querySelectorEval('h1.mob-event-name-heading', 'el => el.innerText')
            event['venueName'] = await page.querySelectorEval('h2.mob-venue-name-details-event', 'el => el.innerText')
            event['attendeesCount'] = await page.querySelectorEval('span#view-count-span', 'el => el.innerText')
        except:
            try:
                event['eventName'] = await page.querySelectorEval('h1.event-name-heading', 'el => el.innerText')
                event['venueName'] = await page.querySelectorEval('h2.venue-name-details-event', 'el => el.innerText')
            except:
                pass

        try:
            event['eventDateAndTime'] = await page.querySelectorEval('div.quick_look_divs:nth-child(2) span.details', 'el => el.innerText')
        except:
            pass

        try:
            event['ticketAmount'] = await page.querySelectorEval('div.quick_look_divs:nth-child(3) span.details', 'el => el.innerText')
        except:
            event['ticketAmount'] = 'Free Entry'

        try:
            event['ticketLink'] = await page.querySelectorEval('a#book_thru_me', 'el => el.href')
        except:
            pass

        try:
            event['highlightImages'] = await page.querySelectorEval('div#image_carousel_web img.img-background-events', 'el => el.src')
        except:
            pass

        try:
            category_elements = await page.querySelectorAllEval('div.quick_look_divs:nth-child(1) span.details h4.category_text', 'nodes => nodes.map(n => n.innerText)')
            event['category'] = ', '.join(category_elements)
        except:
            pass

        try:
            artists = await page.querySelectorAllEval('div#artist h3', 'nodes => nodes.map(n => n.innerText)')
            event['artists'] = ', '.join(artists)
        except:
            pass

        try:
            terms = await page.querySelectorAllEval('#tnc ul li', 'nodes => nodes.map((el, i) => `${i + 1}. ${el.innerText}`)')
            event['policyAndConditions'] = '\n'.join(terms)
            for term in terms:
                if re.search(r'only\s+(\d+)\+', term.lower()):
                    event['minimumAge'] = re.search(r'only\s+(\d+)\+', term.lower()).group(1)
        except:
            pass

        try:
            event['eventDescription'] = await page.querySelectorEval('#desc .event-content-div', 'el => el.innerText')
        except:
            pass

        try:
            address = await page.querySelectorEval('#venue .address p', 'el => el.innerText')
            event['address'] = address
            pincode_match = re.search(r'\b(\d{6})\b', address)
            if pincode_match:
                event['zipcode'] = pincode_match.group(1)

            city_match = re.findall(r'\b([A-Za-z]+)\b', address)
            if city_match:
                event['city'] = city_match[-2] if len(city_match) >= 2 else ''
                event['state'] = city_match[-1]
        except:
            pass

        try:
            event['lat'] = await page.querySelectorEval('#venue_lat', 'el => el.value')
            event['lon'] = await page.querySelectorEval('#venue_lng', 'el => el.value')
            event['geolocation'] = f"{event['lat']},{event['lon']}"
        except:
            pass

        try:
            tags = await page.querySelectorAllEval('div#interest .tags span', 'nodes => nodes.map(n => n.innerText)')
            event['tags'] = ', '.join(tags)
        except:
            pass

    except Exception as e:
        print(f"Error scraping {url}: {e}")

    return event

async def main():
    df_links = pd.read_csv('highape_all_events_links.csv')
    urls = df_links['event_link'].dropna().unique()

    browser = await launch(headless=False, args=['--no-sandbox'])
    page = await browser.newPage()

    all_events = []
    for i, url in enumerate(urls):
        print(f"Scraping {i + 1}/{len(urls)}: {url}")
        event = await scrape_highape_event(page, url)
        all_events.append(event)

    await browser.close()

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"highape_detailed_events_{timestamp}.csv"
    pd.DataFrame(all_events).to_csv(filename, index=False)
    print(f"✅ All events saved to {filename}")

if __name__ == '__main__':
    asyncio.get_event_loop().run_until_complete(main())


In [9]:
import pandas as pd
import uuid
import time
from datetime import datetime
import json
import re
import asyncio
import nest_asyncio
from pyppeteer import launch
import sys
import os
from tqdm import tqdm

nest_asyncio.apply()

CSV_FILE = 'highape_all_events_links.csv'
EVENT_LINK_COLUMN = 'event_link'

def create_empty_event():
    return {
        'externalEventLink': '',
        'eventId': str(uuid.uuid4()),
        'plurEventLink': '',
        'eventName': '',
        'eventDescription': '',
        'eventDateAndTime': '',
        'eventDuration': '',
        'venueName': '',
        'locality': '',
        'address': '',
        'zipcode': '',
        'lat': '',
        'lon': '',
        'ticketAmount': '',
        'guestlistInfo': '',
        'ticketLink': '',
        'supportedLanguages': '',
        'category': '',
        'recurrence': '',
        'attendeesCount': '',
        'joinChatDetails': '',
        'policyAndConditions': '',
        'frequentlyAskedQuestions': '',
        'city': '',
        'state': '',
        'geolocation': '',
        'isRecurring': 'No',
        'pattern': '',
        'eventType': '',
        'subCategory': '',
        'venueId': '',
        'foodAvailable': '',
        'smokingAllowed': '',
        'wheelchairAccess': '',
        'parkingAvailable': '',
        'supportAvailable': '',
        'layout': '',
        'petFriendly': False,
        'alcoholServed': '',
        'minimumAge': '',
        'ticketsAtVenue': '',
        'prohibitedItemList': '',
        'washroomAvailable': '',
        'onlineMenu': '',
        'happyHours': '',
        'danceFloorAvailable': '',
        'poolAvailable': '',
        'isActivityAvailable': '',
        'artists': '',
        'highlightImages': '',
        'highlightVideos': '',
        'galleryImages': '',
        'galleryVideos': '',
        'kidFriendly': False,
        'seatingArrangement': '',
        'entryAllowedFor': '',
        'tags': ''
    }

async def scrape_highape_event(page, url):
    event = create_empty_event()
    event['externalEventLink'] = url

    try:
        await page.setUserAgent('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
        await page.setViewport({'width': 1200, 'height': 800})
        await page.goto(url, {'waitUntil': 'networkidle2', 'timeout': 60000})
        await asyncio.sleep(5)

        # (same scraping logic as before, unchanged...)
        # -- Truncated for brevity --
        
    except Exception as e:
        print(f"❌ Error scraping {url}: {e}")
    return event

async def main():
    # 🔶 Ask user for file input
    user_input = input("👉 Enter the CSV file name to read event links (e.g., highape_all_events_links.csv): ").strip()
    if not user_input.endswith('.csv'):
        print("❌ Please enter a valid CSV filename.")
        sys.exit(1)

    CSV_FILE = user_input
    EVENT_LINK_COLUMN = 'event_link'

    # ✅ Check if file exists
    if not os.path.exists(CSV_FILE):
        print(f"❌ CSV file '{CSV_FILE}' not found. Please ensure it exists in the current directory.")
        sys.exit(1)

    # ✅ Read and validate column
    try:
        df_links = pd.read_csv(CSV_FILE)
    except Exception as e:
        print(f"❌ Failed to read CSV: {e}")
        sys.exit(1)

    if EVENT_LINK_COLUMN not in df_links.columns:
        print(f"❌ Column '{EVENT_LINK_COLUMN}' not found in CSV.")
        print(f"📄 Available columns: {list(df_links.columns)}")
        sys.exit(1)

    urls = df_links[EVENT_LINK_COLUMN].dropna().unique()
    print(f"🔗 Loaded {len(urls)} unique event URLs from '{CSV_FILE}'")

    browser = await launch(headless=False, args=['--no-sandbox'])
    page = await browser.newPage()

    all_events = []
    success_count = 0
    fail_count = 0

    for i, url in enumerate(tqdm(urls, desc="Scraping Events")):
        print(f"➡️  [{i + 1}/{len(urls)}] Scraping: {url}")
        try:
            event = await scrape_highape_event(page, url)
            if event['eventName']:
                success_count += 1
            else:
                fail_count += 1
            all_events.append(event)
        except Exception as e:
            print(f"❌ Failed to scrape {url}: {e}")
            fail_count += 1

    await browser.close()

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"highape_detailed_events_{timestamp}.csv"
    pd.DataFrame(all_events).to_csv(filename, index=False)

    print(f"\n✅ All events saved to {filename}")
    print(f"📊 Summary: {success_count} succeeded, {fail_count} failed")

if __name__ == '__main__':
    asyncio.get_event_loop().run_until_complete(main())


👉 Enter the CSV file name to read event links (e.g., highape_all_events_links.csv):  highape_all_events_bangalore_20250603_120826.csv


🔗 Loaded 294 unique event URLs from 'highape_all_events_bangalore_20250603_120826.csv'


Scraping Events:   0%|                                  | 0/294 [00:00<?, ?it/s]

➡️  [1/294] Scraping: https://highape.com/bangalore/events/talamasca-live-in-bangalore-4qr4eKXGWy?utm_source=allevents_tile


Scraping Events:   0%|                          | 1/294 [00:10<50:21, 10.31s/it]

...

➡️  [293/294] Scraping: https://highape.com/bangalore/events/suit-and-panda-aVWQXNPnrO?utm_source=allevents_list


Scraping Events: 100%|███████████████████████▉| 293/294 [34:41<00:06,  6.86s/it]

➡️  [294/294] Scraping: https://highape.com/bangalore/events/taaruk-raina-live-P5Wo1e5Gk8?utm_source=allevents_list


Scraping Events: 100%|████████████████████████| 294/294 [34:48<00:00,  7.11s/it]



✅ All events saved to highape_detailed_events_20250603_124737.csv
📊 Summary: 0 succeeded, 294 failed


In [17]:
import pandas as pd
import uuid
import asyncio
from datetime import datetime
import json
import re
import nest_asyncio
from pyppeteer import launch
from bs4 import BeautifulSoup

nest_asyncio.apply()

def create_empty_event():
    return {
        'externalEventLink': '',
        'eventId': str(uuid.uuid4()),
        'plurEventLink': '',
        'eventName': '',
        'eventDescription': '',
        'eventDateAndTime': '',
        'eventDuration': '',
        'venueName': '',
        'locality': '',
        'address': '',
        'zipcode': '',
        'lat': '',
        'lon': '',
        'ticketAmount': '',
        'guestlistInfo': '',
        'ticketLink': '',
        'supportedLanguages': '',
        'category': '',
        'recurrence': '',
        'attendeesCount': '',
        'joinChatDetails': '',
        'policyAndConditions': '',
        'frequentlyAskedQuestions': '',
        'city': '',
        'state': '',
        'geolocation': '',
        'isRecurring': 'No',
        'pattern': '',
        'eventType': '',
        'subCategory': '',
        'venueId': '',
        'foodAvailable': '',
        'smokingAllowed': '',
        'wheelchairAccess': '',
        'parkingAvailable': '',
        'supportAvailable': '',
        'layout': '',
        'petFriendly': False,
        'alcoholServed': '',
        'minimumAge': '',
        'ticketsAtVenue': '',
        'prohibitedItemList': '',
        'washroomAvailable': '',
        'onlineMenu': '',
        'happyHours': '',
        'danceFloorAvailable': '',
        'poolAvailable': '',
        'isActivityAvailable': '',
        'artists': '',
        'highlightImages': '',
        'highlightVideos': '',
        'galleryImages': '',
        'galleryVideos': '',
        'kidFriendly': False,
        'seatingArrangement': '',
        'entryAllowedFor': '',
        'tags': '',
        'rawHtml': ''
    }

async def scrape_highape_event(page, url):
    event = create_empty_event()
    event['externalEventLink'] = url

    try:
        await page.setUserAgent('Mozilla/5.0 (Windows NT 10.0; Win64; x64)')
        await page.setViewport({'width': 1200, 'height': 800})
        await page.goto(url, {'waitUntil': 'networkidle2', 'timeout': 60000})
        await asyncio.sleep(5)

        html_content = await page.content()
        event['rawHtml'] = html_content

        # [ Add your event scraping logic here if needed ]
        # soup = BeautifulSoup(html_content, 'html.parser')
        # event['eventName'] = soup.select_one("h1.event-title").text.strip()  # Example

    except Exception as e:
        print(f"❌ Error scraping {url}: {e}")
    return event

async def main():
    # Prompt for a single event link
    url = input("🔗 Enter the HighApe event URL to scrape: ").strip()
    if not url.startswith("http"):
        print("❌ Invalid URL. Please include http(s)://")
        return

    browser = await launch(headless=False, args=['--no-sandbox'])
    page = await browser.newPage()

    print(f"➡️ Scraping: {url}")
    event = await scrape_highape_event(page, url)

    await browser.close()

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"highape_single_event_raw_{timestamp}.csv"
    pd.DataFrame([event]).to_csv(filename, index=False)

    print(f"\n✅ Event scraped and saved to {filename}")

if __name__ == '__main__':
    asyncio.get_event_loop().run_until_complete(main())


🔗 Enter the HighApe event URL to scrape:  https://highape.com/bangalore/events/thursday-ladies-night-yMr7VjRVk9?utm_source=allevents_list


➡️ Scraping: https://highape.com/bangalore/events/thursday-ladies-night-yMr7VjRVk9?utm_source=allevents_list

✅ Event scraped and saved to highape_single_event_raw_20250607_144238.csv


In [19]:
import pandas as pd
import uuid
import time
from datetime import datetime
import json
import re
import asyncio
import nest_asyncio
from pyppeteer import launch

nest_asyncio.apply()

def create_empty_event():
    return {
        'externalEventLink': '',
        'eventId': str(uuid.uuid4()),
        'plurEventLink': '',
        'eventName': '',
        'eventDescription': '',
        'eventDateAndTime': '',
        'eventDuration': '',
        'venueName': '',
        'locality': '',
        'address': '',
        'zipcode': '',
        'lat': '',
        'lon': '',
        'ticketAmount': '',
        'guestlistInfo': '',
        'ticketLink': '',
        'supportedLanguages': '',
        'category': '',
        'recurrence': '',
        'attendeesCount': '',
        'joinChatDetails': '',
        'policyAndConditions': '',
        'frequentlyAskedQuestions': '',
        'city': '',
        'state': '',
        'geolocation': '',
        'isRecurring': 'No',
        'pattern': '',
        'eventType': '',
        'subCategory': '',
        'venueId': '',
        'foodAvailable': '',
        'smokingAllowed': '',
        'wheelchairAccess': '',
        'parkingAvailable': '',
        'supportAvailable': '',
        'layout': '',
        'petFriendly': False,
        'alcoholServed': '',
        'minimumAge': '',
        'ticketsAtVenue': '',
        'prohibitedItemList': '',
        'washroomAvailable': '',
        'onlineMenu': '',
        'happyHours': '',
        'danceFloorAvailable': '',
        'poolAvailable': '',
        'isActivityAvailable': '',
        'artists': '',
        'highlightImages': '',
        'highlightVideos': '',
        'galleryImages': '',
        'galleryVideos': '',
        'kidFriendly': False,
        'seatingArrangement': '',
        'entryAllowedFor': '',
        'tags': '',
        'rawHtml': ''
    }

async def scrape_highape_event(page, url):
    event = create_empty_event()
    event['externalEventLink'] = url

    try:
        await page.setUserAgent('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
        await page.setViewport({'width': 1200, 'height': 800})
        await page.goto(url, {'waitUntil': 'networkidle2', 'timeout': 60000})
        await asyncio.sleep(5)

        # Add raw HTML content
        event['rawHtml'] = await page.content()

        try:
            event['eventName'] = await page.querySelectorEval('h1.mob-event-name-heading', 'el => el.innerText')
            event['venueName'] = await page.querySelectorEval('h2.mob-venue-name-details-event', 'el => el.innerText')
            event['attendeesCount'] = await page.querySelectorEval('span#view-count-span', 'el => el.innerText')
        except:
            try:
                event['eventName'] = await page.querySelectorEval('h1.event-name-heading', 'el => el.innerText')
                event['venueName'] = await page.querySelectorEval('h2.venue-name-details-event', 'el => el.innerText')
            except:
                pass

        try:
            event['eventDateAndTime'] = await page.querySelectorEval('div.quick_look_divs:nth-child(2) span.details', 'el => el.innerText')
        except:
            pass

        try:
            event['ticketAmount'] = await page.querySelectorEval('div.quick_look_divs:nth-child(3) span.details', 'el => el.innerText')
        except:
            event['ticketAmount'] = 'Free Entry'

        try:
            event['ticketLink'] = await page.querySelectorEval('a#book_thru_me', 'el => el.href')
        except:
            pass

        try:
            event['highlightImages'] = await page.querySelectorEval('div#image_carousel_web img.img-background-events', 'el => el.src')
        except:
            pass

        try:
            category_elements = await page.querySelectorAllEval('div.quick_look_divs:nth-child(1) span.details h4.category_text', 'nodes => nodes.map(n => n.innerText)')
            event['category'] = ', '.join(category_elements)
        except:
            pass

        try:
            artists = await page.querySelectorAllEval('div#artist h3', 'nodes => nodes.map(n => n.innerText)')
            event['artists'] = ', '.join(artists)
        except:
            pass

        try:
            terms = await page.querySelectorAllEval('#tnc ul li', 'nodes => nodes.map((el, i) => `${i + 1}. ${el.innerText}`)')
            event['policyAndConditions'] = '\n'.join(terms)
            for term in terms:
                if re.search(r'only\s+(\d+)\+', term.lower()):
                    event['minimumAge'] = re.search(r'only\s+(\d+)\+', term.lower()).group(1)
        except:
            pass

        try:
            event['eventDescription'] = await page.querySelectorEval('#desc .event-content-div', 'el => el.innerText')
        except:
            pass

        try:
            address = await page.querySelectorEval('#venue .address p', 'el => el.innerText')
            event['address'] = address
            pincode_match = re.search(r'\b(\d{6})\b', address)
            if pincode_match:
                event['zipcode'] = pincode_match.group(1)

            city_match = re.findall(r'\b([A-Za-z]+)\b', address)
            if city_match:
                event['city'] = city_match[-2] if len(city_match) >= 2 else ''
                event['state'] = city_match[-1]
        except:
            pass

        try:
            event['lat'] = await page.querySelectorEval('#venue_lat', 'el => el.value')
            event['lon'] = await page.querySelectorEval('#venue_lng', 'el => el.value')
            event['geolocation'] = f"{event['lat']},{event['lon']}"
        except:
            pass

        try:
            tags = await page.querySelectorAllEval('div#interest .tags span', 'nodes => nodes.map(n => n.innerText)')
            event['tags'] = ', '.join(tags)
        except:
            pass

    except Exception as e:
        print(f"Error scraping {url}: {e}")

    return event

async def main():
    url = input("🔗 Enter a HighApe event link to scrape: ").strip()
    if not url.startswith("http"):
        print("❌ Invalid URL.")
        return

    browser = await launch(headless=False, args=['--no-sandbox'])
    page = await browser.newPage()

    print(f"Scraping: {url}")
    event = await scrape_highape_event(page, url)

    await browser.close()

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"highape_single_event_raw_{timestamp}.csv"
    pd.DataFrame([event]).to_csv(filename, index=False)
    print(f"✅ Saved to {filename}")

if __name__ == '__main__':
    asyncio.get_event_loop().run_until_complete(main())


🔗 Enter a HighApe event link to scrape:  https://highape.com/bangalore/events/thursday-ladies-night-yMr7VjRVk9?utm_source=allevents_list


Scraping: https://highape.com/bangalore/events/thursday-ladies-night-yMr7VjRVk9?utm_source=allevents_list
✅ Saved to highape_single_event_raw_20250607_144732.csv


In [21]:
import pandas as pd
import uuid
import asyncio
import nest_asyncio
import re
from datetime import datetime
from pyppeteer import launch
from bs4 import BeautifulSoup

nest_asyncio.apply()

def create_empty_event():
    return {
        'externalEventLink': '',
        'eventId': str(uuid.uuid4()),
        'plurEventLink': '',
        'eventName': '',
        'eventDescription': '',
        'eventDateAndTime': '',
        'eventDuration': '',
        'venueName': '',
        'locality': '',
        'address': '',
        'zipcode': '',
        'lat': '',
        'lon': '',
        'ticketAmount': '',
        'guestlistInfo': '',
        'ticketLink': '',
        'supportedLanguages': '',
        'category': '',
        'recurrence': '',
        'attendeesCount': '',
        'joinChatDetails': '',
        'policyAndConditions': '',
        'frequentlyAskedQuestions': '',
        'city': '',
        'state': '',
        'geolocation': '',
        'isRecurring': 'No',
        'pattern': '',
        'eventType': '',
        'subCategory': '',
        'venueId': '',
        'foodAvailable': '',
        'smokingAllowed': '',
        'wheelchairAccess': '',
        'parkingAvailable': '',
        'supportAvailable': '',
        'layout': '',
        'petFriendly': False,
        'alcoholServed': '',
        'minimumAge': '',
        'ticketsAtVenue': '',
        'prohibitedItemList': '',
        'washroomAvailable': '',
        'onlineMenu': '',
        'happyHours': '',
        'danceFloorAvailable': '',
        'poolAvailable': '',
        'isActivityAvailable': '',
        'artists': '',
        'highlightImages': '',
        'highlightVideos': '',
        'galleryImages': '',
        'galleryVideos': '',
        'kidFriendly': False,
        'seatingArrangement': '',
        'entryAllowedFor': '',
        'tags': '',
        'rawHtml': ''
    }

def extract_event_details_from_html(html, url):
    soup = BeautifulSoup(html, 'html.parser')
    event = create_empty_event()
    event['externalEventLink'] = url
    event['rawHtml'] = html

    def safe_text(selector):
        tag = soup.select_one(selector)
        return tag.get_text(strip=True) if tag else ''

    event['eventName'] = safe_text('h1.mob-event-name-heading') or safe_text('h1.event-name-heading')
    venue_full = safe_text('h2.mob-venue-name-details-event') or safe_text('h2.venue-name-details-event')
    event['venueName'] = re.sub(r'^.*?-\s*', '', venue_full) if venue_full else ''
    event['eventDescription'] = safe_text('#desc .event-content-div')
    event['eventDateAndTime'] = safe_text('div.quick_look_divs:nth-child(2) span.details')

    # Duration extraction
    try:
        parts = event['eventDateAndTime'].split(' - ')
        if len(parts) == 2:
            start_str, end_str = parts
            start_time = re.search(r'\d{1,2}:\d{2}\s*[APMapm]{2}', start_str)
            end_time = re.search(r'\d{1,2}:\d{2}\s*[APMapm]{2}', end_str)
            if start_time and end_time:
                fmt = "%I:%M %p"
                start = datetime.strptime(start_time.group(), fmt)
                end = datetime.strptime(end_time.group(), fmt)
                if end <= start:
                    end = end.replace(hour=end.hour + 12)
                duration_ms = int((end - start).total_seconds() * 1000)
                event['eventDuration'] = str(duration_ms)
    except:
        pass

    event['ticketAmount'] = safe_text('div.quick_look_divs:nth-child(3) span.details') or 'Free Entry'
    event['ticketLink'] = soup.select_one('a#book_thru_me[href]')['href'] if soup.select_one('a#book_thru_me[href]') else ''
    event['category'] = ', '.join([el.get_text(strip=True) for el in soup.select('div.quick_look_divs h4.category_text')])
    event['tags'] = ', '.join([el.get_text(strip=True) for el in soup.select('#interest .tags span')])
    event['artists'] = ', '.join([el.get_text(strip=True) for el in soup.select('#artist h3')])

    # Terms & Conditions
    terms = soup.select('#tnc ul li')
    event['policyAndConditions'] = '\n'.join([f"{i+1}. {el.get_text(strip=True)}" for i, el in enumerate(terms)])
    for t in terms:
        txt = t.get_text().lower()
        if 'only' in txt and '+' in txt:
            match = re.search(r'only\s+(\d+)\+', txt)
            if match:
                event['minimumAge'] = match.group(1)
        if 'no alcohol' in txt:
            event['alcoholServed'] = 'No'
        elif 'alcohol' in txt:
            event['alcoholServed'] = 'Yes'
        if 'no smoking' in txt:
            event['smokingAllowed'] = 'No'
        elif 'smoking' in txt:
            event['smokingAllowed'] = 'Yes'
        if 'kids allowed' in txt:
            event['kidFriendly'] = True
        if 'wheelchair' in txt:
            event['wheelchairAccess'] = 'Yes'
        if 'parking' in txt:
            event['parkingAvailable'] = 'Yes'
        if 'food' in txt:
            event['foodAvailable'] = 'Yes'

    # Address and geo
    address_text = safe_text('#venue .address p')
    event['address'] = address_text
    pincode_match = re.search(r'\b(\d{6})\b', address_text)
    if pincode_match:
        event['zipcode'] = pincode_match.group(1)

    words = re.findall(r'[A-Za-z]+', address_text)
    if len(words) >= 3:
        event['city'] = words[-3]
        event['state'] = words[-2]
        event['locality'] = words[-4] if len(words) >= 4 else ''

    event['lat'] = soup.select_one('#venue_lat')['value'] if soup.select_one('#venue_lat') else ''
    event['lon'] = soup.select_one('#venue_lng')['value'] if soup.select_one('#venue_lng') else ''
    if event['lat'] and event['lon']:
        event['geolocation'] = f"{event['lat']},{event['lon']}"

    # Media
    event['highlightImages'] = soup.select_one('#image_carousel_web img.img-background-events')['src'] if soup.select_one('#image_carousel_web img.img-background-events') else ''

    return event

async def main():
    url = input("🔗 Enter the HighApe event link: ").strip()
    if not url.startswith("http"):
        print("❌ Invalid URL.")
        return

    browser = await launch(headless=False, args=['--no-sandbox'])
    page = await browser.newPage()
    await page.setUserAgent('Mozilla/5.0 (Windows NT 10.0; Win64; x64)')
    await page.setViewport({'width': 1200, 'height': 800})

    print(f"🔄 Scraping: {url}")
    await page.goto(url, {'waitUntil': 'networkidle2', 'timeout': 60000})
    await asyncio.sleep(5)
    raw_html = await page.content()
    await browser.close()

    event = extract_event_details_from_html(raw_html, url)
    df = pd.DataFrame([event])

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output = f"highape_single_event_enriched_{timestamp}.csv"
    df.to_csv(output, index=False, quoting=1)
    print(f"✅ Data saved to {output}")

if __name__ == "__main__":
    asyncio.get_event_loop().run_until_complete(main())


🔗 Enter the HighApe event link:  https://highape.com/bangalore/events/tipsyaana-shaam-N5k2L4JLWd?utm_source=allevents_list


🔄 Scraping: https://highape.com/bangalore/events/tipsyaana-shaam-N5k2L4JLWd?utm_source=allevents_list
✅ Data saved to highape_single_event_enriched_20250607_150346.csv


In [23]:
import pandas as pd
import uuid
import asyncio
import nest_asyncio
import re
import json
import os
from datetime import datetime
from pyppeteer import launch
from bs4 import BeautifulSoup

nest_asyncio.apply()

OUTPUT_DIR = "event_json_files"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def create_empty_event():
    return {
        'externalEventLink': '',
        'eventId': str(uuid.uuid4()),
        'plurEventLink': '',
        'eventName': '',
        'eventDescription': '',
        'eventDateAndTime': '',
        'eventDuration': '',
        'venueName': '',
        'locality': '',
        'address': '',
        'zipcode': '',
        'lat': '',
        'lon': '',
        'ticketAmount': '',
        'guestlistInfo': '',
        'ticketLink': '',
        'supportedLanguages': '',
        'category': '',
        'recurrence': '',
        'attendeesCount': '',
        'joinChatDetails': '',
        'policyAndConditions': '',
        'frequentlyAskedQuestions': '',
        'city': '',
        'state': '',
        'geolocation': '',
        'isRecurring': 'No',
        'pattern': '',
        'eventType': '',
        'subCategory': '',
        'venueId': '',
        'foodAvailable': '',
        'smokingAllowed': '',
        'wheelchairAccess': '',
        'parkingAvailable': '',
        'supportAvailable': '',
        'layout': '',
        'petFriendly': False,
        'alcoholServed': '',
        'minimumAge': '',
        'ticketsAtVenue': '',
        'prohibitedItemList': '',
        'washroomAvailable': '',
        'onlineMenu': '',
        'happyHours': '',
        'danceFloorAvailable': '',
        'poolAvailable': '',
        'isActivityAvailable': '',
        'artists': '',
        'highlightImages': '',
        'highlightVideos': '',
        'galleryImages': '',
        'galleryVideos': '',
        'kidFriendly': False,
        'seatingArrangement': '',
        'entryAllowedFor': '',
        'tags': '',
        'Plur_json_format': '',
        'jsonFilePath': ''
    }

def build_plur_json(event):
    def to_int(val): return int(val) if str(val).isdigit() else 0
    def to_float(val): return float(val) if re.match(r'^-?\d+(?:\.\d+)?$', str(val)) else None

    return {
        "eventName": event["eventName"],
        "eventDescription": event["eventDescription"],
        "eventDateAndTime": event["eventDateAndTime"],
        "eventDuration": to_int(event["eventDuration"]),
        "venue": {
            "venueName": event["venueName"],
            "locality": event["locality"],
            "address": event["address"],
            "city": event["city"],
            "state": event["state"],
            "zipcode": event["zipcode"],
            "geolocation": {
                "lat": to_float(event["lat"]),
                "lon": to_float(event["lon"]),
            },
            "layout": event.get("layout", "")
        },
        "highlightImageLinks": [event["highlightImages"]] if event["highlightImages"] else [],
        "galleryImageLinks": [event["highlightImages"]] if event["highlightImages"] else [],
        "ticketAmount": to_int(event["ticketAmount"]),
        "ticketLink": event["ticketLink"],
        "supportedLanguages": ["English"],
        "category": event["category"],
        "subCategory": event["subCategory"],
        "eventType": event["eventType"],
        "eventFeatures": {
            "foodAvailable": event["foodAvailable"] == "Yes",
            "smokingAllowed": event["smokingAllowed"] == "Yes",
            "wheelchairAccess": event["wheelchairAccess"] == "Yes",
            "parkingAvailable": event["parkingAvailable"] == "Yes",
            "supportAvailable": event["supportAvailable"] == "Yes",
            "petFriendly": event["petFriendly"],
            "alcoholServed": event["alcoholServed"] == "Yes",
            "minimumAge": to_int(event["minimumAge"]),
            "ticketsAtVenue": event["ticketsAtVenue"] == "Yes",
            "washroomAvailable": event["washroomAvailable"] == "Yes",
            "danceFloorAvailable": event["danceFloorAvailable"] == "Yes",
            "poolAvailable": event["poolAvailable"] == "Yes"
        },
        "artists": event["artists"].split(", ") if event["artists"] else [],
        "sharableEventOgImageLink": event["highlightImages"],
        "attendeesCount": to_int(event["attendeesCount"]),
        "likesCount": 0,
        "joinChatDetails": {
            "joinChatLink": "",
            "provider": "",
            "isEnabled": False
        },
        "policyAndConditions": event["policyAndConditions"].split("\n") if event["policyAndConditions"] else [],
        "frequentlyAskedQuestions": []
    }

async def scrape_highape_event(url):
    browser = await launch(headless=False, args=['--no-sandbox'])
    page = await browser.newPage()
    await page.setUserAgent('Mozilla/5.0')
    await page.goto(url, {'waitUntil': 'networkidle2', 'timeout': 60000})
    await asyncio.sleep(5)
    raw_html = await page.content()
    await browser.close()

    soup = BeautifulSoup(raw_html, 'html.parser')
    event = create_empty_event()
    event['externalEventLink'] = url
    event['rawHtml'] = raw_html  # for .json file

    def sel(selector): return soup.select_one(selector)
    def txt(selector): return sel(selector).get_text(strip=True) if sel(selector) else ''
    def href(selector): return sel(selector)['href'] if sel(selector) else ''

    event['eventName'] = txt('h1.mob-event-name-heading') or txt('h1.event-name-heading')
    venue_full = txt('h2.mob-venue-name-details-event') or txt('h2.venue-name-details-event')
    event['venueName'] = re.sub(r'^.*?-\s*', '', venue_full)
    event['eventDescription'] = txt('#desc .event-content-div')
    event['eventDateAndTime'] = txt('div.quick_look_divs:nth-child(2) span.details')
    event['ticketAmount'] = txt('div.quick_look_divs:nth-child(3) span.details')
    event['ticketLink'] = href('a#book_thru_me')
    event['category'] = ', '.join([e.get_text(strip=True) for e in soup.select('div.quick_look_divs h4.category_text')])
    event['artists'] = ', '.join([e.get_text(strip=True) for e in soup.select('#artist h3')])
    event['highlightImages'] = sel('#image_carousel_web img.img-background-events')['src'] if sel('#image_carousel_web img.img-background-events') else ''

    address = txt('#venue .address p')
    event['address'] = address
    zip_match = re.search(r'\b(\d{6})\b', address)
    if zip_match: event['zipcode'] = zip_match.group(1)
    parts = re.findall(r'[A-Za-z]+', address)
    if len(parts) >= 4:
        event['locality'] = parts[-4]
        event['city'] = parts[-3]
        event['state'] = parts[-2]

    event['lat'] = sel('#venue_lat')['value'] if sel('#venue_lat') else ''
    event['lon'] = sel('#venue_lng')['value'] if sel('#venue_lng') else ''
    event['geolocation'] = f"{event['lat']},{event['lon']}"

    # Terms
    terms = soup.select('#tnc ul li')
    event['policyAndConditions'] = '\n'.join([f"{i+1}. {t.get_text(strip=True)}" for i, t in enumerate(terms)])
    for t in terms:
        txt_lower = t.get_text().lower()
        if '21+' in txt_lower: event['minimumAge'] = "21"
        if 'alcohol' in txt_lower: event['alcoholServed'] = 'Yes'
        if 'smoking' in txt_lower: event['smokingAllowed'] = 'Yes'
        if 'parking' in txt_lower: event['parkingAvailable'] = 'Yes'
        if 'food' in txt_lower: event['foodAvailable'] = 'Yes'
        if 'wheelchair' in txt_lower: event['wheelchairAccess'] = 'Yes'

    return event

async def main():
    url = input("🔗 Enter HighApe event URL: ").strip()
    event = await scrape_highape_event(url)

    # Save rawHtml to JSON file
    json_file = os.path.join(OUTPUT_DIR, f"event_{event['eventId']}.json")
    with open(json_file, 'w', encoding='utf-8') as f:
        json.dump({
            "eventId": event['eventId'],
            "externalEventLink": event['externalEventLink'],
            "rawHtml": event['rawHtml']
        }, f, ensure_ascii=False, indent=2)
    event['jsonFilePath'] = json_file

    # Build structured JSON and inject into CSV
    plur_json = build_plur_json(event)
    event['Plur_json_format'] = json.dumps(plur_json, ensure_ascii=False)

    # Remove rawHtml from the CSV row
    del event['rawHtml']

    # Save CSV
    df = pd.DataFrame([event])
    df.to_csv("highape_event_output.csv", index=False, quoting=1)
    print("✅ CSV saved as: highape_event_output.csv")
    print(f"✅ JSON with raw HTML saved as: {json_file}")

if __name__ == "__main__":
    asyncio.get_event_loop().run_until_complete(main())


🔗 Enter HighApe event URL:  https://highape.com/bangalore/events/tipsyaana-shaam-N5k2L4JLWd?utm_source=allevents_list


✅ CSV saved as: highape_event_output.csv
✅ JSON with raw HTML saved as: event_json_files/event_ac85dead-6262-4014-82d1-a9305608612c.json


In [31]:
import pandas as pd
import uuid
import asyncio
import nest_asyncio
import re
import json
import os
from datetime import datetime
from pyppeteer import launch
from bs4 import BeautifulSoup

nest_asyncio.apply()

OUTPUT_DIR = "event_json_files"
os.makedirs(OUTPUT_DIR, exist_ok=True)

SUMMARY = {
    "total": 0,
    "success": 0,
    "failed": 0,
    "failed_links": []
}

def create_empty_event():
    return {
        'externalEventLink': '', 'eventId': str(uuid.uuid4()), 'plurEventLink': '',
        'eventName': '', 'eventDescription': '', 'eventDateAndTime': '', 'eventDuration': '',
        'venueName': '', 'locality': '', 'address': '', 'zipcode': '', 'lat': '', 'lon': '',
        'ticketAmount': '', 'guestlistInfo': '', 'ticketLink': '', 'supportedLanguages': '',
        'category': '', 'recurrence': '', 'attendeesCount': '', 'joinChatDetails': '',
        'policyAndConditions': '', 'frequentlyAskedQuestions': '', 'city': '', 'state': '',
        'geolocation': '', 'isRecurring': 'No', 'pattern': '', 'eventType': '', 'subCategory': '',
        'venueId': '', 'foodAvailable': '', 'smokingAllowed': '', 'wheelchairAccess': '',
        'parkingAvailable': '', 'supportAvailable': '', 'layout': '', 'petFriendly': False,
        'alcoholServed': '', 'minimumAge': '', 'ticketsAtVenue': '', 'prohibitedItemList': '',
        'washroomAvailable': '', 'onlineMenu': '', 'happyHours': '', 'danceFloorAvailable': '',
        'poolAvailable': '', 'isActivityAvailable': '', 'artists': '', 'highlightImages': '',
        'highlightVideos': '', 'galleryImages': '', 'galleryVideos': '', 'kidFriendly': False,
        'seatingArrangement': '', 'entryAllowedFor': '', 'tags': '',
        'Plur_json_format': '', 'jsonFilePath': ''
    }

def build_plur_json(event):
    def to_int(val): return int(val) if str(val).isdigit() else 0
    def to_float(val): return float(val) if re.match(r'^-?\d+(?:\.\d+)?$', str(val)) else None

    return {
        "eventName": event["eventName"],
        "eventDescription": event["eventDescription"],
        "eventDateAndTime": event["eventDateAndTime"],
        "eventDuration": to_int(event["eventDuration"]),
        "venue": {
            "venueName": event["venueName"],
            "locality": event["locality"],
            "address": event["address"],
            "city": event["city"],
            "state": event["state"],
            "zipcode": event["zipcode"],
            "geolocation": {
                "lat": to_float(event["lat"]),
                "lon": to_float(event["lon"]),
            },
            "layout": event.get("layout", "")
        },
        "highlightImageLinks": [event["highlightImages"]] if event["highlightImages"] else [],
        "galleryImageLinks": [event["highlightImages"]] if event["highlightImages"] else [],
        "ticketAmount": to_int(event["ticketAmount"]),
        "ticketLink": event["ticketLink"],
        "supportedLanguages": ["English"],
        "category": event["category"],
        "subCategory": event["subCategory"],
        "eventType": event["eventType"],
        "eventFeatures": {
            "foodAvailable": event["foodAvailable"] == "Yes",
            "smokingAllowed": event["smokingAllowed"] == "Yes",
            "wheelchairAccess": event["wheelchairAccess"] == "Yes",
            "parkingAvailable": event["parkingAvailable"] == "Yes",
            "supportAvailable": event["supportAvailable"] == "Yes",
            "petFriendly": event["petFriendly"],
            "alcoholServed": event["alcoholServed"] == "Yes",
            "minimumAge": to_int(event["minimumAge"]),
            "ticketsAtVenue": event["ticketsAtVenue"] == "Yes",
            "washroomAvailable": event["washroomAvailable"] == "Yes",
            "danceFloorAvailable": event["danceFloorAvailable"] == "Yes",
            "poolAvailable": event["poolAvailable"] == "Yes"
        },
        "artists": event["artists"].split(", ") if event["artists"] else [],
        "sharableEventOgImageLink": event["highlightImages"],
        "attendeesCount": to_int(event["attendeesCount"]),
        "likesCount": 0,
        "joinChatDetails": {
            "joinChatLink": "",
            "provider": "",
            "isEnabled": False
        },
        "policyAndConditions": event["policyAndConditions"].split("\n") if event["policyAndConditions"] else [],
        "frequentlyAskedQuestions": []
    }

async def scrape_highape_event(url):
    event = create_empty_event()
    event['externalEventLink'] = url
    try:
        browser = await launch(headless=False, args=['--no-sandbox'])
        page = await browser.newPage()
        await page.setUserAgent('Mozilla/5.0')
        await page.goto(url, {'waitUntil': 'domcontentloaded', 'timeout': 60000})
        await asyncio.sleep(5)
        raw_html = await page.content()
        await browser.close()

        soup = BeautifulSoup(raw_html, 'html.parser')
        event['rawHtml'] = raw_html

        def sel(selector): return soup.select_one(selector)
        def txt(selector): return sel(selector).get_text(strip=True) if sel(selector) else ''
        def href(selector): return sel(selector)['href'] if sel(selector) else ''

        event['eventName'] = txt('h1.mob-event-name-heading') or txt('h1.event-name-heading')
        venue_full = txt('h2.mob-venue-name-details-event') or txt('h2.venue-name-details-event')
        event['venueName'] = re.sub(r'^.*?-\s*', '', venue_full)
        event['eventDescription'] = txt('#desc .event-content-div')
        event['eventDateAndTime'] = txt('div.quick_look_divs:nth-child(2) span.details')
        event['ticketAmount'] = txt('div.quick_look_divs:nth-child(3) span.details') or 'Free Entry'
        event['ticketLink'] = href('a#book_thru_me')
        event['category'] = ', '.join([e.get_text(strip=True) for e in soup.select('div.quick_look_divs h4.category_text')])
        event['artists'] = ', '.join([e.get_text(strip=True) for e in soup.select('#artist h3')])
        event['highlightImages'] = sel('#image_carousel_web img.img-background-events')['src'] if sel('#image_carousel_web img.img-background-events') else ''

        address = txt('#venue .address p')
        event['address'] = address
        zip_match = re.search(r'\b(\d{6})\b', address)
        if zip_match: event['zipcode'] = zip_match.group(1)
        parts = re.findall(r'[A-Za-z]+', address)
        if len(parts) >= 4:
            event['locality'] = parts[-4]
            event['city'] = parts[-3]
            event['state'] = parts[-2]

        event['lat'] = sel('#venue_lat')['value'] if sel('#venue_lat') else ''
        event['lon'] = sel('#venue_lng')['value'] if sel('#venue_lng') else ''
        event['geolocation'] = f"{event['lat']},{event['lon']}"

        terms = soup.select('#tnc ul li')
        event['policyAndConditions'] = '\n'.join([f"{i+1}. {t.get_text(strip=True)}" for i, t in enumerate(terms)])
        for t in terms:
            txt_lower = t.get_text().lower()
            if '21+' in txt_lower: event['minimumAge'] = "21"
            if 'alcohol' in txt_lower: event['alcoholServed'] = 'Yes'
            if 'smoking' in txt_lower: event['smokingAllowed'] = 'Yes'
            if 'parking' in txt_lower: event['parkingAvailable'] = 'Yes'
            if 'food' in txt_lower: event['foodAvailable'] = 'Yes'
            if 'wheelchair' in txt_lower: event['wheelchairAccess'] = 'Yes'

        json_file = os.path.join(OUTPUT_DIR, f"event_{event['eventId']}.json")
        with open(json_file, 'w', encoding='utf-8') as f:
            json.dump({"eventId": event['eventId'], "externalEventLink": url, "rawHtml": raw_html}, f, indent=2)
        event['jsonFilePath'] = json_file

        plur_json = build_plur_json(event)
        event['Plur_json_format'] = json.dumps(plur_json, ensure_ascii=False, indent=2)

        del event['rawHtml']
        SUMMARY["success"] += 1
        return event
    except Exception as e:
        SUMMARY["failed"] += 1
        SUMMARY["failed_links"].append(url)
        print(f"❌ Failed to scrape: {url}\n{e}")
        return None

async def main():
    input_csv = input("📄 Enter the input CSV file name: ").strip()
    column_name = input("🔍 Enter the column name containing event links: ").strip()

    if not os.path.exists(input_csv):
        print(f"❌ File not found: {input_csv}")
        return

    df_links = pd.read_csv(input_csv)
    if column_name not in df_links.columns:
        print(f"❌ Column '{column_name}' not found in CSV")
        return

    urls = df_links[column_name].dropna().unique()[:5]  # Limit to first 5 for test
    SUMMARY["total"] = len(urls)
    print(f"🚀 Starting to scrape {len(urls)} events...\n")

    enriched_events = []
    for url in urls:
        event = await scrape_highape_event(url)
        if event:
            enriched_events.append(event)

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_csv = f"highape_events_enriched_{timestamp}.csv"
    pd.DataFrame(enriched_events).to_csv(output_csv, index=False, quoting=1)

    print(f"\n✅ Scraping completed. CSV saved as: {output_csv}")
    print(f"📊 Summary: {SUMMARY['success']} succeeded, {SUMMARY['failed']} failed out of {SUMMARY['total']}")
    if SUMMARY['failed']:
        print("❗ Failed links:")
        for link in SUMMARY['failed_links']:
            print(f"- {link}")

if __name__ == "__main__":
    asyncio.get_event_loop().run_until_complete(main())


📄 Enter the input CSV file name:  /Users/shiv/Documents/Jupyter/HighApe/highape_all_events_bangalore_mumbai_hyderabad_ncr_pune_goa_20250607_143542.csv
🔍 Enter the column name containing event links:  event_link


🚀 Starting to scrape 5 events...


✅ Scraping completed. CSV saved as: highape_events_enriched_20250607_154906.csv
📊 Summary: 5 succeeded, 0 failed out of 5


In [35]:
import pandas as pd
import uuid
import asyncio
import nest_asyncio
import re
import json
import os
from datetime import datetime, timedelta
from pyppeteer import launch
from bs4 import BeautifulSoup

nest_asyncio.apply()

OUTPUT_DIR = "event_json_files"
os.makedirs(OUTPUT_DIR, exist_ok=True)

SUMMARY = {
    "total": 0,
    "success": 0,
    "failed": 0,
    "failed_links": []
}

ALL_PLUR_JSONS = []

def create_empty_event():
    return {
        'externalEventLink': '', 'eventId': str(uuid.uuid4()), 'plurEventLink': '',
        'eventName': '', 'eventDescription': '', 'eventDateAndTime': '', 'eventDuration': '',
        'venueName': '', 'locality': '', 'address': '', 'zipcode': '', 'lat': '', 'lon': '',
        'ticketAmount': '', 'guestlistInfo': '', 'ticketLink': '', 'supportedLanguages': '',
        'category': '', 'recurrence': '', 'attendeesCount': '', 'joinChatDetails': '',
        'policyAndConditions': '', 'frequentlyAskedQuestions': '', 'city': '', 'state': '',
        'geolocation': '', 'isRecurring': 'No', 'pattern': '', 'eventType': '', 'subCategory': '',
        'venueId': '', 'foodAvailable': '', 'smokingAllowed': '', 'wheelchairAccess': '',
        'parkingAvailable': '', 'supportAvailable': '', 'layout': '', 'petFriendly': False,
        'alcoholServed': '', 'minimumAge': '', 'ticketsAtVenue': '', 'prohibitedItemList': '',
        'washroomAvailable': '', 'onlineMenu': '', 'happyHours': '', 'danceFloorAvailable': '',
        'poolAvailable': '', 'isActivityAvailable': '', 'artists': '', 'highlightImages': '',
        'highlightVideos': '', 'galleryImages': '', 'galleryVideos': '', 'kidFriendly': False,
        'seatingArrangement': '', 'entryAllowedFor': '', 'tags': '',
        'Plur_json_format': '', 'jsonFilePath': ''
    }

def build_plur_json(event):
    def to_int(val): return int(val) if str(val).isdigit() else 0
    def to_float(val): return float(val) if re.match(r'^-?\d+(?:\.\d+)?$', str(val)) else None

    return {
        "eventName": event["eventName"],
        "eventDescription": event["eventDescription"],
        "eventDateAndTime": event["eventDateAndTime"],
        "eventDuration": to_int(event["eventDuration"]),
        "venue": {
            "venueName": event["venueName"],
            "locality": event["locality"],
            "address": event["address"],
            "city": event["city"],
            "state": event["state"],
            "zipcode": event["zipcode"],
            "geolocation": {
                "lat": to_float(event["lat"]),
                "lon": to_float(event["lon"]),
            },
            "layout": event.get("layout", "")
        },
        "highlightImageLinks": [event["highlightImages"]] if event["highlightImages"] else [],
        "galleryImageLinks": [event["highlightImages"]] if event["highlightImages"] else [],
        "ticketAmount": to_int(event["ticketAmount"]),
        "ticketLink": event["ticketLink"],
        "supportedLanguages": ["English"],
        "category": event["category"],
        "subCategory": event["subCategory"],
        "eventType": event["eventType"],
        "eventFeatures": {
            "foodAvailable": event["foodAvailable"] == "Yes",
            "smokingAllowed": event["smokingAllowed"] == "Yes",
            "wheelchairAccess": event["wheelchairAccess"] == "Yes",
            "parkingAvailable": event["parkingAvailable"] == "Yes",
            "supportAvailable": event["supportAvailable"] == "Yes",
            "petFriendly": event["petFriendly"],
            "alcoholServed": event["alcoholServed"] == "Yes",
            "minimumAge": to_int(event["minimumAge"]),
            "ticketsAtVenue": event["ticketsAtVenue"] == "Yes",
            "washroomAvailable": event["washroomAvailable"] == "Yes",
            "danceFloorAvailable": event["danceFloorAvailable"] == "Yes",
            "poolAvailable": event["poolAvailable"] == "Yes"
        },
        "artists": event["artists"].split(", ") if event["artists"] else [],
        "sharableEventOgImageLink": event["highlightImages"],
        "attendeesCount": to_int(event["attendeesCount"]),
        "likesCount": 0,
        "joinChatDetails": {
            "joinChatLink": "",
            "provider": "",
            "isEnabled": False
        },
        "policyAndConditions": event["policyAndConditions"].split("\n") if event["policyAndConditions"] else [],
        "frequentlyAskedQuestions": []
    }

def parse_event_duration(text):
    try:
        date_time_match = re.search(r'(\d{1,2} \w+)(?: - (\d{1,2} \w+))?(?: \| (\d{1,2}:\d{2} [APMapm]{2}))?', text)
        if not date_time_match:
            return text, ''

        start_date = date_time_match.group(1)
        end_date = date_time_match.group(2) or start_date
        time_str = date_time_match.group(3) or "6:00 PM"

        now_year = datetime.now().year
        start = datetime.strptime(f"{start_date} {now_year} {time_str}", "%d %B %Y %I:%M %p")
        end = datetime.strptime(f"{end_date} {now_year} {time_str}", "%d %B %Y %I:%M %p")
        duration_ms = int((end - start).total_seconds() * 1000)
        iso_str = start.isoformat() + "Z"
        return iso_str, str(duration_ms)
    except Exception:
        return text, ''

async def save_event_json_batch():
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_path = f"highape_events_structured_{timestamp}.json"
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(ALL_PLUR_JSONS, f, ensure_ascii=False, indent=2)
    print(f"📦 Batch structured JSON saved as: {output_path}")


async def main():
    input_csv = input("📄 Enter the input CSV file name: ").strip()
    column_name = input("🔍 Enter the column name containing event links: ").strip()

    if not os.path.exists(input_csv):
        print(f"❌ File not found: {input_csv}")
        return

    df_links = pd.read_csv(input_csv)
    if column_name not in df_links.columns:
        print(f"❌ Column '{column_name}' not found in CSV")
        return

    urls = df_links[column_name].dropna().unique()[:5]
    SUMMARY["total"] = len(urls)
    print(f"🚀 Starting to scrape {len(urls)} events...\n")

    # You would call your scraping function here, and store results into ALL_PLUR_JSONS
    # Then call save_event_json_batch() at the end

if __name__ == "__main__":
    asyncio.get_event_loop().run_until_complete(main())


📄 Enter the input CSV file name:  /Users/shiv/Documents/Jupyter/HighApe/highape_all_events_bangalore_mumbai_hyderabad_ncr_pune_goa_20250607_143542.csv
🔍 Enter the column name containing event links:  event_link


🚀 Starting to scrape 5 events...



In [37]:
import pandas as pd
import uuid
import asyncio
import nest_asyncio
import re
import json
import os
from datetime import datetime, timedelta
from pyppeteer import launch
from bs4 import BeautifulSoup

nest_asyncio.apply()

OUTPUT_DIR = "event_json_files"
os.makedirs(OUTPUT_DIR, exist_ok=True)

SUMMARY = {
    "total": 0,
    "success": 0,
    "failed": 0,
    "failed_links": []
}

ALL_PLUR_JSONS = []

def create_empty_event():
    return {
        'externalEventLink': '', 'eventId': str(uuid.uuid4()), 'plurEventLink': '',
        'eventName': '', 'eventDescription': '', 'eventDateAndTime': '', 'eventDuration': '',
        'venueName': '', 'locality': '', 'address': '', 'zipcode': '', 'lat': '', 'lon': '',
        'ticketAmount': '', 'guestlistInfo': '', 'ticketLink': '', 'supportedLanguages': '',
        'category': '', 'recurrence': '', 'attendeesCount': '', 'joinChatDetails': '',
        'policyAndConditions': '', 'frequentlyAskedQuestions': '', 'city': '', 'state': '',
        'geolocation': '', 'isRecurring': 'No', 'pattern': '', 'eventType': '', 'subCategory': '',
        'venueId': '', 'foodAvailable': '', 'smokingAllowed': '', 'wheelchairAccess': '',
        'parkingAvailable': '', 'supportAvailable': '', 'layout': '', 'petFriendly': False,
        'alcoholServed': '', 'minimumAge': '', 'ticketsAtVenue': '', 'prohibitedItemList': '',
        'washroomAvailable': '', 'onlineMenu': '', 'happyHours': '', 'danceFloorAvailable': '',
        'poolAvailable': '', 'isActivityAvailable': '', 'artists': '', 'highlightImages': '',
        'highlightVideos': '', 'galleryImages': '', 'galleryVideos': '', 'kidFriendly': False,
        'seatingArrangement': '', 'entryAllowedFor': '', 'tags': '',
        'Plur_json_format': '', 'jsonFilePath': ''
    }

def build_plur_json(event):
    def to_int(val): return int(val) if str(val).isdigit() else 0
    def to_float(val): return float(val) if re.match(r'^-?\d+(?:\.\d+)?$', str(val)) else None

    return {
        "eventName": event["eventName"],
        "eventDescription": event["eventDescription"],
        "eventDateAndTime": event["eventDateAndTime"],
        "eventDuration": to_int(event["eventDuration"]),
        "venue": {
            "venueName": event["venueName"],
            "locality": event["locality"],
            "address": event["address"],
            "city": event["city"],
            "state": event["state"],
            "zipcode": event["zipcode"],
            "geolocation": {
                "lat": to_float(event["lat"]),
                "lon": to_float(event["lon"]),
            },
            "layout": event.get("layout", "")
        },
        "highlightImageLinks": [event["highlightImages"]] if event["highlightImages"] else [],
        "galleryImageLinks": [event["highlightImages"]] if event["highlightImages"] else [],
        "ticketAmount": to_int(event["ticketAmount"]),
        "ticketLink": event["ticketLink"],
        "supportedLanguages": ["English"],
        "category": event["category"],
        "subCategory": event["subCategory"],
        "eventType": event["eventType"],
        "eventFeatures": {
            "foodAvailable": event["foodAvailable"] == "Yes",
            "smokingAllowed": event["smokingAllowed"] == "Yes",
            "wheelchairAccess": event["wheelchairAccess"] == "Yes",
            "parkingAvailable": event["parkingAvailable"] == "Yes",
            "supportAvailable": event["supportAvailable"] == "Yes",
            "petFriendly": event["petFriendly"],
            "alcoholServed": event["alcoholServed"] == "Yes",
            "minimumAge": to_int(event["minimumAge"]),
            "ticketsAtVenue": event["ticketsAtVenue"] == "Yes",
            "washroomAvailable": event["washroomAvailable"] == "Yes",
            "danceFloorAvailable": event["danceFloorAvailable"] == "Yes",
            "poolAvailable": event["poolAvailable"] == "Yes"
        },
        "artists": event["artists"].split(", ") if event["artists"] else [],
        "sharableEventOgImageLink": event["highlightImages"],
        "attendeesCount": to_int(event["attendeesCount"]),
        "likesCount": 0,
        "joinChatDetails": {
            "joinChatLink": "",
            "provider": "",
            "isEnabled": False
        },
        "policyAndConditions": event["policyAndConditions"].split("\n") if event["policyAndConditions"] else [],
        "frequentlyAskedQuestions": []
    }

def parse_event_duration(text):
    try:
        date_time_match = re.search(r'(\d{1,2} \w+)(?: - (\d{1,2} \w+))?(?: \| (\d{1,2}:\d{2} [APMapm]{2}))?', text)
        if not date_time_match:
            return text, ''

        start_date = date_time_match.group(1)
        end_date = date_time_match.group(2) or start_date
        time_str = date_time_match.group(3) or "6:00 PM"

        now_year = datetime.now().year
        start = datetime.strptime(f"{start_date} {now_year} {time_str}", "%d %B %Y %I:%M %p")
        end = datetime.strptime(f"{end_date} {now_year} {time_str}", "%d %B %Y %I:%M %p")
        duration_ms = int((end - start).total_seconds() * 1000)
        iso_str = start.isoformat() + "Z"
        return iso_str, str(duration_ms)
    except Exception:
        return text, ''

async def scrape_highape_event(url):
    event = create_empty_event()
    event['externalEventLink'] = url
    try:
        browser = await launch(headless=False, args=['--no-sandbox'])
        page = await browser.newPage()
        await page.setUserAgent('Mozilla/5.0')
        await page.goto(url, {'waitUntil': 'domcontentloaded', 'timeout': 60000})
        await asyncio.sleep(5)
        raw_html = await page.content()
        await browser.close()

        # Place HTML parsing logic here and update event fields

        iso_str, duration_ms = parse_event_duration(event.get("eventDateAndTime", ""))
        event["eventDateAndTime"] = iso_str
        event["eventDuration"] = duration_ms

        plur_json = build_plur_json(event)
        event['Plur_json_format'] = json.dumps(plur_json, ensure_ascii=False, indent=2)
        ALL_PLUR_JSONS.append(plur_json)

        file_path = os.path.join(OUTPUT_DIR, f"event_{event['eventId']}.json")
        event['jsonFilePath'] = file_path
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump({"eventId": event['eventId'], "externalEventLink": url, "rawHtml": raw_html}, f, indent=2)

        SUMMARY["success"] += 1
        return event
    except Exception as e:
        SUMMARY["failed"] += 1
        SUMMARY["failed_links"].append(url)
        print(f"❌ Failed to scrape: {url}\n{e}")
        return None

async def save_event_json_batch():
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_path = f"highape_events_structured_{timestamp}.json"
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(ALL_PLUR_JSONS, f, ensure_ascii=False, indent=2)
    print(f"📦 Batch structured JSON saved as: {output_path}")

async def main():
    input_csv = input("📄 Enter the input CSV file name: ").strip()
    column_name = input("🔍 Enter the column name containing event links: ").strip()

    if not os.path.exists(input_csv):
        print(f"❌ File not found: {input_csv}")
        return

    df_links = pd.read_csv(input_csv)
    if column_name not in df_links.columns:
        print(f"❌ Column '{column_name}' not found in CSV")
        return

    urls = df_links[column_name].dropna().unique()[:5]
    SUMMARY["total"] = len(urls)
    print(f"🚀 Starting to scrape {len(urls)} events...\n")

    for url in urls:
        await scrape_highape_event(url)

    await save_event_json_batch()

    print(f"\n✅ Scraping completed.")
    print(f"📊 Summary: {SUMMARY['success']} succeeded, {SUMMARY['failed']} failed out of {SUMMARY['total']}")
    if SUMMARY['failed']:
        print("❗ Failed links:")
        for link in SUMMARY['failed_links']:
            print(f"- {link}")

if __name__ == "__main__":
    asyncio.get_event_loop().run_until_complete(main())


📄 Enter the input CSV file name:  /Users/shiv/Documents/Jupyter/HighApe/highape_all_events_bangalore_mumbai_hyderabad_ncr_pune_goa_20250607_143542.csv
🔍 Enter the column name containing event links:  event_link


🚀 Starting to scrape 5 events...

📦 Batch structured JSON saved as: highape_events_structured_20250607_160657.json

✅ Scraping completed.
📊 Summary: 5 succeeded, 0 failed out of 5


In [39]:
import pandas as pd
import uuid
import asyncio
import nest_asyncio
import re
import json
import os
from datetime import datetime, timedelta
from pyppeteer import launch
from bs4 import BeautifulSoup

nest_asyncio.apply()

RUN_TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")
OUTPUT_DIR = f"event_json_files/run_{RUN_TIMESTAMP}"
os.makedirs(OUTPUT_DIR, exist_ok=True)

SUMMARY = {
    "total": 0,
    "success": 0,
    "failed": 0,
    "failed_links": []
}

ALL_PLUR_JSONS = []

def create_empty_event():
    return {
        'externalEventLink': '', 'eventId': str(uuid.uuid4()), 'plurEventLink': '',
        'eventName': '', 'eventDescription': '', 'eventDateAndTime': '', 'eventDuration': '',
        'venueName': '', 'locality': '', 'address': '', 'zipcode': '', 'lat': '', 'lon': '',
        'ticketAmount': '', 'guestlistInfo': '', 'ticketLink': '', 'supportedLanguages': '',
        'category': '', 'recurrence': '', 'attendeesCount': '', 'joinChatDetails': '',
        'policyAndConditions': '', 'frequentlyAskedQuestions': '', 'city': '', 'state': '',
        'geolocation': '', 'isRecurring': 'No', 'pattern': '', 'eventType': '', 'subCategory': '',
        'venueId': '', 'foodAvailable': '', 'smokingAllowed': '', 'wheelchairAccess': '',
        'parkingAvailable': '', 'supportAvailable': '', 'layout': '', 'petFriendly': False,
        'alcoholServed': '', 'minimumAge': '', 'ticketsAtVenue': '', 'prohibitedItemList': '',
        'washroomAvailable': '', 'onlineMenu': '', 'happyHours': '', 'danceFloorAvailable': '',
        'poolAvailable': '', 'isActivityAvailable': '', 'artists': '', 'highlightImages': '',
        'highlightVideos': '', 'galleryImages': '', 'galleryVideos': '', 'kidFriendly': False,
        'seatingArrangement': '', 'entryAllowedFor': '', 'tags': '',
        'Plur_json_format': '', 'jsonFilePath': ''
    }

def build_plur_json(event):
    def to_int(val): return int(val) if str(val).isdigit() else 0
    def to_float(val): return float(val) if re.match(r'^-?\d+(?:\.\d+)?$', str(val)) else None

    return {
        "eventName": event["eventName"],
        "eventDescription": event["eventDescription"],
        "eventDateAndTime": event["eventDateAndTime"],
        "eventDuration": to_int(event["eventDuration"]),
        "venue": {
            "venueName": event["venueName"],
            "locality": event["locality"],
            "address": event["address"],
            "city": event["city"],
            "state": event["state"],
            "zipcode": event["zipcode"],
            "geolocation": {
                "lat": to_float(event["lat"]),
                "lon": to_float(event["lon"]),
            },
            "layout": event.get("layout", "")
        },
        "highlightImageLinks": [event["highlightImages"]] if event["highlightImages"] else [],
        "galleryImageLinks": [event["highlightImages"]] if event["highlightImages"] else [],
        "ticketAmount": to_int(event["ticketAmount"]),
        "ticketLink": event["ticketLink"],
        "supportedLanguages": ["English"],
        "category": event["category"],
        "subCategory": event["subCategory"],
        "eventType": event["eventType"],
        "eventFeatures": {
            "foodAvailable": event["foodAvailable"] == "Yes",
            "smokingAllowed": event["smokingAllowed"] == "Yes",
            "wheelchairAccess": event["wheelchairAccess"] == "Yes",
            "parkingAvailable": event["parkingAvailable"] == "Yes",
            "supportAvailable": event["supportAvailable"] == "Yes",
            "petFriendly": event["petFriendly"],
            "alcoholServed": event["alcoholServed"] == "Yes",
            "minimumAge": to_int(event["minimumAge"]),
            "ticketsAtVenue": event["ticketsAtVenue"] == "Yes",
            "washroomAvailable": event["washroomAvailable"] == "Yes",
            "danceFloorAvailable": event["danceFloorAvailable"] == "Yes",
            "poolAvailable": event["poolAvailable"] == "Yes"
        },
        "artists": event["artists"].split(", ") if event["artists"] else [],
        "sharableEventOgImageLink": event["highlightImages"],
        "attendeesCount": to_int(event["attendeesCount"]),
        "likesCount": 0,
        "joinChatDetails": {
            "joinChatLink": "",
            "provider": "",
            "isEnabled": False
        },
        "policyAndConditions": event["policyAndConditions"].split("\n") if event["policyAndConditions"] else [],
        "frequentlyAskedQuestions": []
    }

async def scrape_highape_event(url):
    event = create_empty_event()
    event['externalEventLink'] = url
    try:
        browser = await launch(headless=False, args=['--no-sandbox'])
        page = await browser.newPage()
        await page.setUserAgent('Mozilla/5.0')
        await page.goto(url, {'waitUntil': 'domcontentloaded', 'timeout': 60000})
        await asyncio.sleep(5)
        raw_html = await page.content()
        await browser.close()

        soup = BeautifulSoup(raw_html, 'html.parser')
        event['rawHtml'] = raw_html

        def sel(selector): return soup.select_one(selector)
        def txt(selector): return sel(selector).get_text(strip=True) if sel(selector) else ''
        def href(selector): return sel(selector)['href'] if sel(selector) else ''

        event['eventName'] = txt('h1.mob-event-name-heading') or txt('h1.event-name-heading')
        venue_full = txt('h2.mob-venue-name-details-event') or txt('h2.venue-name-details-event')
        event['venueName'] = re.sub(r'^.*?-\s*', '', venue_full)
        event['eventDescription'] = txt('#desc .event-content-div')
        event['eventDateAndTime'] = txt('div.quick_look_divs:nth-child(2) span.details')
        event['ticketAmount'] = txt('div.quick_look_divs:nth-child(3) span.details') or 'Free Entry'
        event['ticketLink'] = href('a#book_thru_me')
        event['category'] = ', '.join([e.get_text(strip=True) for e in soup.select('div.quick_look_divs h4.category_text')])
        event['artists'] = ', '.join([e.get_text(strip=True) for e in soup.select('#artist h3')])
        event['highlightImages'] = sel('#image_carousel_web img.img-background-events')['src'] if sel('#image_carousel_web img.img-background-events') else ''

        address = txt('#venue .address p')
        event['address'] = address
        zip_match = re.search(r'\b(\d{6})\b', address)
        if zip_match: event['zipcode'] = zip_match.group(1)
        parts = re.findall(r'[A-Za-z]+', address)
        if len(parts) >= 4:
            event['locality'] = parts[-4]
            event['city'] = parts[-3]
            event['state'] = parts[-2]

        event['lat'] = sel('#venue_lat')['value'] if sel('#venue_lat') else ''
        event['lon'] = sel('#venue_lng')['value'] if sel('#venue_lng') else ''
        event['geolocation'] = f"{event['lat']},{event['lon']}"

        terms = soup.select('#tnc ul li')
        event['policyAndConditions'] = '\n'.join([f"{i+1}. {t.get_text(strip=True)}" for i, t in enumerate(terms)])
        for t in terms:
            txt_lower = t.get_text().lower()
            if '21+' in txt_lower: event['minimumAge'] = "21"
            if 'alcohol' in txt_lower: event['alcoholServed'] = 'Yes'
            if 'smoking' in txt_lower: event['smokingAllowed'] = 'Yes'
            if 'parking' in txt_lower: event['parkingAvailable'] = 'Yes'
            if 'food' in txt_lower: event['foodAvailable'] = 'Yes'
            if 'wheelchair' in txt_lower: event['wheelchairAccess'] = 'Yes'

        json_file = os.path.join(OUTPUT_DIR, f"event_{event['eventId']}.json")
        with open(json_file, 'w', encoding='utf-8') as f:
            json.dump({"eventId": event['eventId'], "externalEventLink": url, "rawHtml": raw_html}, f, indent=2)
        event['jsonFilePath'] = json_file

        plur_json = build_plur_json(event)
        event['Plur_json_format'] = json.dumps(plur_json, ensure_ascii=False, indent=2)
        ALL_PLUR_JSONS.append(plur_json)

        del event['rawHtml']
        SUMMARY["success"] += 1
        return event
    except Exception as e:
        SUMMARY["failed"] += 1
        SUMMARY["failed_links"].append(url)
        print(f"❌ Failed to scrape: {url}\n{e}")
        return None

async def main():
    input_csv = input("📄 Enter the input CSV file name: ").strip()
    column_name = input("🔍 Enter the column name containing event links: ").strip()

    if not os.path.exists(input_csv):
        print(f"❌ File not found: {input_csv}")
        return

    df_links = pd.read_csv(input_csv)
    if column_name not in df_links.columns:
        print(f"❌ Column '{column_name}' not found in CSV")
        return

    urls = df_links[column_name].dropna().unique()
    SUMMARY["total"] = len(urls)
    print(f"🚀 Starting to scrape {len(urls)} events...\n")

    enriched_events = []
    for url in urls:
        event = await scrape_highape_event(url)
        if event:
            enriched_events.append(event)

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_csv = f"highape_events_enriched_{timestamp}.csv"
    pd.DataFrame(enriched_events).to_csv(output_csv, index=False, quoting=1)

    with open(f"highape_events_structured_{timestamp}.json", "w", encoding="utf-8") as f:
        json.dump(ALL_PLUR_JSONS, f, ensure_ascii=False, indent=2)

    print(f"\n✅ Scraping completed. CSV saved as: {output_csv}")
    print(f"📊 Summary: {SUMMARY['success']} succeeded, {SUMMARY['failed']} failed out of {SUMMARY['total']}")
    if SUMMARY['failed']:
        print("❗ Failed links:")
        for link in SUMMARY['failed_links']:
            print(f"- {link}")

if __name__ == "__main__":
    asyncio.get_event_loop().run_until_complete(main())


📄 Enter the input CSV file name:  /Users/shiv/Documents/Jupyter/HighApe/highape_all_events_bangalore_mumbai_hyderabad_ncr_pune_goa_20250607_143542.csv
🔍 Enter the column name containing event links:  event_link


🚀 Starting to scrape 356 events...

❌ Failed to scrape: https://highape.com/bangalore/events/centigrade-club-nights-ft-axl-stace--7thjunepangeo-6nkPOpOVwL?utm_source=allevents_list
Protocol Error (Runtime.callFunctionOn): Session closed. Most likely the page has been closed.

✅ Scraping completed. CSV saved as: highape_events_enriched_20250607_171657.csv
📊 Summary: 355 succeeded, 1 failed out of 356
❗ Failed links:
- https://highape.com/bangalore/events/centigrade-club-nights-ft-axl-stace--7thjunepangeo-6nkPOpOVwL?utm_source=allevents_list
